In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install distance
!pip install fuzzywuzzy

     |████████████████████████████████| 184kB 23.5MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp37-none-any.whl size=16261 sha256=bcb3cae5b8b4c624037cf28ea8e1a191c843d49af646275be9d9cf55d342a15d
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [5]:
import os
import re
import csv
import numpy as np 
import pandas as pd 
from string import punctuation
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import pickle 
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import sys
sys.path.append('/content/drive/MyDrive/Project')
import feature_engineering

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
class Test:
  def __init__(self,tokenizer_name = '/content/drive/MyDrive/Project/tokenizer.pickle',ques_dict_name = '/content/drive/MyDrive/Project/q_dict.pickle',\
               ss_name = '/content/drive/MyDrive/Project/ss.pickle',model_name = '/content/drive/MyDrive/Project/royal_model.h5'):
    
    with open(tokenizer_name, 'rb') as handle:
      self.tokenizer = pickle.load(handle)
    self.word_index = self.tokenizer.word_index
    print('Tokenizer Loaded')

    with open(ques_dict_name, 'rb') as handle:
      self.ques_dict = pickle.load(handle)
    print('Question Dict Loaded')

    with open(ss_name, 'rb') as handle:
      self.ss = pickle.load(handle)
    print('Standard Scalar Loaded') 

    self.model = load_model(model_name)
    print('Model Loaded')

    self.test_question1 = []
    self.test_question2 = []
    self.featureEngineering = feature_engineering.FeatureEngineering()

  def text_to_wordlist(self,text):
    
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"coronavirus","corona virus 2019",text)
    text = re.sub(r"covid-19","corona virus 2019",text)
    text = re.sub(r"covid - 19","corona virus 2019",text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text
  
  def tokenization(self,df):

    for text in df.question1.values:
      self.test_question1.append(self.text_to_wordlist(text))
      
    for text in df.question2.values:
      self.test_question2.append(self.text_to_wordlist(text))

    self.test_question1 = self.tokenizer.texts_to_sequences(self.test_question1)  
    self.test_question2 = self.tokenizer.texts_to_sequences(self.test_question2)

    print('Tokenization Done....')

    MAX_SEQUENCE_LENGTH = 60  
    self.test_question1 = pad_sequences(self.test_question1, maxlen=MAX_SEQUENCE_LENGTH)  
    self.test_question2 = pad_sequences(self.test_question2, maxlen=MAX_SEQUENCE_LENGTH)  
    print('Shape of test data vtensor:', self.test_question1.shape)

  def feature_extraction(self,df):
    questions = pd.concat([df[['question1', 'question2']]], axis=0).reset_index(drop='index')
    for i in range(questions.shape[0]):
            self.ques_dict[questions.question1[i]].add(questions.question2[i])
            self.ques_dict[questions.question2[i]].add(questions.question1[i])
    df['q1_q2_intersect'] = df.apply(lambda row: len(set(self.ques_dict.get(row[1])).intersection(set(self.ques_dict.get(row[2])))), axis=1, raw=True)
    df['q1_freq'] = df.apply(lambda row: len(self.ques_dict.get(row[1])), axis=1, raw=True)
    df['q2_freq'] = df.apply(lambda row: len(self.ques_dict.get(row[2])), axis=1, raw=True)

    print('Feature Extraction Completed..')
    return df

  def leak_calculator(self,df,df_nlp):
    df_nlp = df_nlp.fillna(0)
    df['test_id'] = df_nlp['test_id']

    df = df.merge(df_nlp,on='test_id',how='left')
    leak = df.drop(['test_id','question1','question2'], axis=1)
    leak = self.ss.transform(leak)
    
    print('Leak Data Completed..')
    return leak


  def predict(self,question1,question2):
    self.test_question1 = []
    self.test_question2 = []
    df = pd.DataFrame(data=[[0,question1,question2]] , columns=['test_id','question1','question2'])
    self.tokenization(df)
    df = self.feature_extraction(df)
    df_nlp = self.featureEngineering.two_question(question1,question2)
    leaks = self.leak_calculator(df,df_nlp)
    preds = self.model.predict([self.test_question1, self.test_question2, leaks], batch_size=8192, verbose=1,)
    preds += self.model.predict([self.test_question2, self.test_question1, leaks], batch_size=8192, verbose=1)
    return preds[0][0]

In [11]:
test = Test() # run only once

Tokenizer Loaded
Question Dict Loaded
Standard Scalar Loaded
Model Loaded


In [26]:
question1 = 'What will be the long term impact of demonetization of rs 500 and rs 1000?'
question2 = 'What can be poker the reasons for discontinuation of 500 and 1000 rupee?'

In [27]:
print('Predicted probability:- ',test.predict(question1=question1 , question2=question2))

Tokenization Done....
Shape of test data vtensor: (1, 60)
Feature Extraction Completed..
Extracting Token Features...
Extracting Fuzzy Features..
Extracting Distance Features..
- word_mover_dis done...
- embedding done...
- spatial distance done
Leak Data Completed..
1/1 [==============================] - 0s 99ms/step
Predicted probability:-  0.068573
